# Importing Modules

In [ ]:
!pip install llama-index
!pip install llama-index-embeddings-huggingface
!pip install peft
!pip install auto-gptq
!pip install optimum
!pip install bitsandbytes
!pip  install fitz
!pip install frontend
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.8/173.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.5/383.5 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing insta

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex, Document
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from sentence_transformers import SentenceTransformer
from google.colab import drive
import fitz
import os

# Loading Data And Model

In [ ]:
# setting the device to pc
import torch
if torch.cuda.is_available:
        device = torch.device("cuda")
else:
    device = 'cpu'
device

device(type='cuda')

In [ ]:
#(https://huggingface.co/spaces/mteb/leaderboard)
Settings.embed_model = HuggingFaceEmbedding(model_name="thenlper/gte-large", device=device) # alternative model HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

Settings.llm = None
Settings.chunk_size = 512
Settings.chunk_overlap = 40

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

LLM is explicitly disabled. Using MockLLM.


In [ ]:
# Funtion to extarct text frim the pdf file
def extract_text_from_pdfs(pdf_files):
    documents = []
    for pdf_file in pdf_files:
        doc = fitz.open(pdf_file)
        text = ""
        for page in doc:
            text += page.get_text()
        documents.append({"text": text, "filename": pdf_file})
    return documents

In [ ]:
# make the document chunks
def chunk_documents(documents, chunk_size, chunk_overlap):
    chunks = []
    for doc in documents:
        text = doc['text']
        for start in range(0, len(text), chunk_size - chunk_overlap):
            end = min(start + chunk_size, len(text))
            chunks.append({"text": text[start:end], "filename": doc['filename']})
    return chunks

In [ ]:
# Mount the content from drive
drive.mount('/content/drive')
drive_path ='/content/drive/MyDrive/Baymax/'
dataset_path = drive_path + "Dataset_RAG/"

Mounted at /content/drive


In [ ]:
pdf_files = [dataset_path+file for file in os.listdir(dataset_path) if file.endswith('.pdf')] #list of book names
documents = extract_text_from_pdfs(pdf_files) #Extracted documents(books)

In [ ]:
#chunk settings

chunk_size = 512
chunk_overlap = 40
chunked_documents = chunk_documents(documents, chunk_size, chunk_overlap)
chunked_documents = [Document(text=doc['text']) for doc in chunked_documents]

index = VectorStoreIndex.from_documents(chunked_documents)

In [ ]:
top_k = 3 #Best 3 values where return

#creating Vector Index Retriever Object
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=top_k,
)

In [ ]:
# setting up retriver query Engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)],
)

In [ ]:
# Runs till the user exit the code
while True:
  query = input("Enter your query: ") #user query
  response = query_engine.query(query)
  context = "Context:\n"
  for i in range(top_k):
      context = context + response.source_nodes[i].text + "\n\n"

  print(context)
  if(input("break??")=='y'):
    break

Enter your query: Hypertension is a condition where the force of the blood against the artery walls is too high, often leading to health problems such as heart disease."
Context:
lic pressure greater than 90 
mm Hg. HTN causes the left ventricle to enlarge (hyper­
trophy) as a result of increased work. Some cases of HTN 
are secondary to other disorders, such as kidney malfunc­
tion or endocrine disturbance, but most of the time, the 
causes are unknown, a condition described as primary, or 
essential, HTN. 
Treatment of Hypertension 
Even though there is much individual variation in blood 
pressure, physicians have established guidelines for the 
diagnosis and treatment of hyperten

the 
diagnosis and treatment of hypertension. Blood pressure 
readings of 120/80 to 139/89 describe prehypertension, a 
warning sign for the future development of high blood pres­
sure. This condition is treated by lifestyle modifications such 
as increased physical exercise, a low-salt, low-fat diet, and 

# Save the RAG

In [ ]:
save_path = drive_path + "RAG_MODEL/" #model save path
index.storage_context.persist(persist_dir=save_path)

# Load the RAG and Test

In [ ]:
load_path = drive_path + "RAG_MODEL/" #model load path
storage_context = StorageContext.from_defaults(persist_dir=load_path)
loaded_index = load_index_from_storage(storage_context=storage_context)

In [ ]:
top_k = 3
#creating Vector Index Retriever Object
retriever_load = VectorIndexRetriever(
    index=loaded_index,
    similarity_top_k=top_k,
)

# setting up retriver query Engine
query_engine_load = RetrieverQueryEngine(
    retriever=retriever_load,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)],
)

In [ ]:
# Runs till the user exit the code
while True:
  query = input("Enter your query: ")
  response = query_engine_load.query(query)
  context = "Context:\n"
  for i in range(top_k):
      context = context + response.source_nodes[i].text + "\n\n"

  print(context)
  if(input("break??")=='y'):
    break

Enter your query: how to find kidney stones
Context:
abnormal pH of urine, urinary sta­
sis, and metabolic imbalances.  The stones generally form 
in the kidney and may move to the bladder).  This results 
in great pain,  termed renal colic,  and obstruction that can 
promote infection and cause hydronephrosis,  collection of 
urine in the renal pelvis. 
Because they are radiopaque, stones can usually be seen 
on simple radiographs of the abdomen. Stones may dissolve 
and pass out of the body on their own. If not, they may 
be removed surgically,  in a litho

ths ago. Her gynecologist  
warned her that calcium could be a problem for people  
who are “stone formers.” Helen was unaware that she might  
be at risk. An IV urogram showed a right staghorn calculus.  
The diagnosis was further confirmed by a renal ultrasound.  
A renal flow scan showed normal perfusion and no obstruc-
tion. Kidney function was 37% on the right and 63% on the  
left. The pain became intermittent, and Helen had